[View in Colaboratory](https://colab.research.google.com/github/josd/eye/blob/master/transduction/observation_prediction.ipynb)

# Transduction from observation to prediction

## Introduction

What is [Transduction (machine learning)](https://en.wikipedia.org/wiki/Transduction_(machine_learning%29):

> In logic, statistical inference, and supervised learning, transduction or transductive inference is reasoning from observed, specific (training) cases to specific (test) cases. In contrast, induction is reasoning from observed training cases to general rules, which are then applied to the test cases. The distinction is most interesting in cases where the predictions of the transductive model are not achievable by any inductive model. Note that this is caused by transductive inference on different test sets producing mutually inconsistent predictions.

What is the Tensor2Tensor [Transformer model](https://github.com/tensorflow/tensor2tensor/blob/master/tensor2tensor/models/transformer.py):

> The Transformer model consists of an encoder and a decoder. Both are stacks
of self-attention layers followed by feed-forward layers. This model yields
good results on a number of problems, especially in NLP and machine translation.
See "Attention Is All You Need" (https://arxiv.org/abs/1706.03762) for the full
description of the model and the results obtained with its early version.

![Transformer model](https://pbs.twimg.com/media/DCKhefrUMAE9stK.jpg)

In [5]:
# Preparation

! pip install -U tensor2tensor
! curl -o observation_prediction.sh http://josd.github.io/eye/transduction/observation_prediction.sh
! curl -o observation_prediction.py http://josd.github.io/eye/transduction/observation_prediction.py
! curl -o __init__.py http://josd.github.io/eye/transduction/__init__.py
! curl -o sample.observation http://josd.github.io/eye/transduction/sample.observation
! chmod +x observation_prediction.sh
#% rm -fr ~/t2t_train/observation_prediction/transformer-transformer_small/
#% rm -fr ~/t2t_data/

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
Requirement already up-to-date: tensor2tensor in /usr/local/lib/python3.6/dist-packages (1.6.0)
Requirement not upgraded as not directly required: gevent in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.2.2)
Requirement not upgraded as not directly required: future in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.16.0)
Requirement not upgraded as not directly required: google-api-python-client in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.6.6)
Requirement not upgraded as not directly required: numpy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (1.14.2)
Requirement not upgraded as not directly required: bz2file in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.98)
Requirement not upgraded as not directly required: scipy in /usr/local/lib/python3.6/dist-packages (from tensor2tensor) (0.19.1)
Requirement not upgraded as not dir

In [6]:
# See the observation_prediction problem

! pygmentize -g observation_prediction.py

import random
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.utils import registry

@registry.register_problem
class ObservationPrediction(text_problems.Text2TextProblem):
  """Transduction from observation to prediction."""

  @property
  def approx_vocab_size(self):
    return 2**14  # ~16k

  @property
  def is_generate_per_split(self):
    # generate_data will shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 50% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 5,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 5,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    del data_dir
    del tmp_dir
    del dataset_split

    for n in range(10000):
      # wind turbine size factor
      size_f

In [7]:
# See the observation_prediction script

! pygmentize -g observation_prediction.sh

#!/bin/bash
PROBLEM=observation_prediction
MODEL=transformer
HPARAMS=transformer_small

USER_DIR=$PWD
DATA_DIR=$HOME/t2t_data
TMP_DIR=/tmp/t2t_datagen
TRAIN_DIR=$HOME/t2t_train/$PROBLEM/$MODEL-$HPARAMS

mkdir -p $DATA_DIR $TMP_DIR $TRAIN_DIR

# Generate data
t2t-datagen \
  --data_dir=$DATA_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --tmp_dir=$TMP_DIR

# Train
t2t-trainer \
  --data_dir=$DATA_DIR \
  --eval_steps=3 \
  --hparams_set=$HPARAMS \
  --local_eval_frequency=100 \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --train_steps=2000 \
  --worker_gpu_memory_fraction=0.75

# Decode
t2t-decoder \
  --data_dir=$DATA_DIR \
  --decode_from_file=sample.observation \
  --decode_hparams="beam_size=3,alpha=0.6,return_beams=True,write_beam_scores=True" \
  --decode_to_file=sample.prediction \
  --hparams_set=$HPARAMS \
  --model=$MODEL \
  --output_dir=$TRAIN_DIR \
  --problem=$PROBLEM \
  --t2t_usr_dir=$USER_DIR \
  --wor

In [8]:
# Run the observation_prediction script

! ./observation_prediction.sh

/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
[2018-04-26 21:50:29,277] Importing user module content from path /
INFO:tensorflow:Generating problems:
    observation:
      * observation_prediction
[2018-04-26 21:50:29,279] Generating problems:
    observation:
      * observation_prediction
INFO:tensorflow:Generating data for observation_prediction.
[2018-04-26 21:50:29,280] Generating data for observation_prediction.
INFO:tensorflow:Found vocab file: /content/t2t_data/vocab.observation_prediction.16384.subwords
[2018-04-26 21:50:29,280] Found vocab file: /content/t2t_data/vocab.observation_prediction.16384.subwords
INFO:tensorflow:Skipping generator because outputs files e

INFO:tensorflow:Done calling model_fn.
[2018-04-26 21:50:48,950] Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
[2018-04-26 21:50:48,952] Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
[2018-04-26 21:50:50,580] Graph was finalized.
2018-04-26 21:50:50.673017: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:898] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2018-04-26 21:50:50.673405: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1344] Found device 0 with properties: 
name: Tesla K80 major: 3 minor: 7 memoryClockRate(GHz): 0.8235
pciBusID: 0000:00:04.0
totalMemory: 11.17GiB freeMemory: 11.10GiB
2018-04-26 21:50:50.673442: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1423] Adding visible gpu devices: 0
2018-04-26 21:50:51.007701: I tensorflow/core/common_runtime/gpu/gpu_device.cc:911] Device interconnect StreamExecutor with strength 1 edge matrix:


/usr/local/lib/python3.6/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module content from path /
[2018-04-26 21:51:35,821] Importing user module content from path /
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
[2018-04-26 21:51:35,981] From /usr/local/lib/python3.6/dist-packages/tensor2tensor/utils/trainer_lib.py:152: RunConfig.__init__ (from tensorflow.contrib.learn.python.learn.estimators.run_config) is deprecated and will be removed in a future version.
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
[2018-04-26 21:51:35,981] schedule=continuous_train_and_ev

In [9]:
# See the transductions
# For each target the top 3 is shown with their scores (log probability)

! pygmentize -g sample.observation
print("->-")
! pygmentize -g sample.prediction

A_PERSON with weight 74 kg and height 179 cm
A_TURBINE with size factor 4 and subjected to windspeed 62 km/h
->-
A_PERSON has BMI class N	-0.34	A_PERSON has BMI class U	-4.58	A_PERSON has BMI class O	-5.07
A_TURBINE producing 9533 kW	-0.36	A_TURBINE producing 16682 kW	-3.11	A_TURBINE producing 16208 kW	-5.05
